In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

In [17]:
data = pd.read_csv('D:\data1.csv')

In [33]:
x = data.iloc[:, 2:]
y_age = data.iloc[:, 1]
y_sex = data.iloc[:, 0]

In [34]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_re, y_re = oversample.fit_resample(x, y_sex)

In [35]:
sc= MinMaxScaler()
pdata=pd.DataFrame(sc.fit_transform(x_re),columns=x_re.columns,index=x_re.index)
x_new = pdata.iloc[:, :]

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x_new , y_re, test_size=0.2 , random_state=1)

In [37]:
reg = LogisticRegression()
reg.fit(x_train,y_train)
lr_pred=reg.predict(x_test)# # linear reg
print("Classification Report is:\n",classification_report(y_test,lr_pred))

Classification Report is:
               precision    recall  f1-score   support

           0       0.70      0.55      0.62        29
           1       0.55      0.70      0.62        23

    accuracy                           0.62        52
   macro avg       0.62      0.62      0.62        52
weighted avg       0.63      0.62      0.62        52



In [39]:
mlp=MLPClassifier(activation='tanh',shuffle=True,random_state=2023)
mlp.fit(x_train , y_train )
y_pred=mlp.predict(x_test)
accuracy_score(y_test,y_pred)
mlp_clf = MLPClassifier(hidden_layer_sizes=(150,100,50),
                        max_iter = 300,activation = 'relu',
                        solver = 'adam')

mlp_clf.fit(x_train, y_train)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.62      0.68        29
           1       0.61      0.74      0.67        23

    accuracy                           0.67        52
   macro avg       0.68      0.68      0.67        52
weighted avg       0.69      0.67      0.67        52



In [38]:
RFC = RandomForestClassifier()
n_estimators = [500]
max_features = [ 'auto']
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=RFC, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
best_model = grid_search.fit(x_train, y_train)
rf_pred=best_model.predict(x_test)
print("Classification Report is:\n",classification_report(y_test,rf_pred))

Classification Report is:
               precision    recall  f1-score   support

           0       0.75      0.72      0.74        29
           1       0.67      0.70      0.68        23

    accuracy                           0.71        52
   macro avg       0.71      0.71      0.71        52
weighted avg       0.71      0.71      0.71        52



In [40]:
y_re_tst = y_re.to_numpy()
y_re_tst.shape

y_re_tst = y_re_tst.reshape(-1,1)
print(y_re_tst.shape)

encoder = OneHotEncoder()
a =encoder.fit(y_re_tst)

y_encoded = a.transform(y_re_tst).toarray()


(256, 1)


In [145]:
from random import randint
x_train, x_test, y_train, y_test = train_test_split(x_new , y_encoded, test_size=0.2 , random_state=randint(100, 1000))
print(x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape)

(204, 12) (52, 12) (204, 2) (52, 2)


In [146]:
model2 = keras.Sequential()

model2.add(Dense(1024, kernel_initializer ="uniform",activation = "relu", input_dim=12))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(512, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(256, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(128, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(64, kernel_initializer = "uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(32, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(16, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(8, kernel_initializer ="uniform",activation = "softmax"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(2))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

In [147]:
model2.compile(optimizer= "SGD", loss = "MSE", metrics = ["accuracy"])

In [148]:
print("Fit model on training data")
history = model2.fit(x_train, y_train, batch_size=70, epochs=100)

Fit model on training data
Epoch 1/100
3/3 [==============================] - 10s 57ms/step - loss: 1.2920 - accuracy: 0.6029
Epoch 2/100
3/3 [==============================] - 0s 53ms/step - loss: 0.8828 - accuracy: 0.7745
Epoch 3/100
3/3 [==============================] - 0s 55ms/step - loss: 0.7453 - accuracy: 0.8480
Epoch 4/100
3/3 [==============================] - 0s 58ms/step - loss: 0.6468 - accuracy: 0.8775
Epoch 5/100
3/3 [==============================] - 0s 56ms/step - loss: 0.5994 - accuracy: 0.8529
Epoch 6/100
3/3 [==============================] - 0s 53ms/step - loss: 0.5262 - accuracy: 0.9020
Epoch 7/100
3/3 [==============================] - 0s 68ms/step - loss: 0.4622 - accuracy: 0.9559
Epoch 8/100
3/3 [==============================] - 0s 61ms/step - loss: 0.4604 - accuracy: 0.9314
Epoch 9/100
3/3 [==============================] - 0s 51ms/step - loss: 0.4237 - accuracy: 0.9314
Epoch 10/100
3/3 [==============================] - 0s 55ms/step - loss: 0.3781 - accuracy

In [149]:
history = model2.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
7/7 [==============================] - 2s 277ms/step - loss: 0.0386 - accuracy: 0.9755 - val_loss: 0.2473 - val_accuracy: 0.7500
Epoch 2/100
7/7 [==============================] - 0s 47ms/step - loss: 0.0509 - accuracy: 0.9608 - val_loss: 0.2468 - val_accuracy: 0.7115
Epoch 3/100
7/7 [==============================] - 0s 65ms/step - loss: 0.0467 - accuracy: 0.9706 - val_loss: 0.2463 - val_accuracy: 0.6731
Epoch 4/100
7/7 [==============================] - 0s 49ms/step - loss: 0.0711 - accuracy: 0.9216 - val_loss: 0.2457 - val_accuracy: 0.7115
Epoch 5/100
7/7 [==============================] - 0s 45ms/step - loss: 0.0521 - accuracy: 0.9608 - val_loss: 0.2446 - val_accuracy: 0.6923
Epoch 6/100
7/7 [==============================] - 0s 49ms/step - loss: 0.0669 - accuracy: 0.9118 - val_loss: 0.2453 - val_accuracy: 0.5962
Epoch 7/100
7/7 [==============================] - 0s 47ms/step - loss: 0.0925 - accuracy: 0.9118 - val_loss: 0.2438 - val_accuracy: 0.6731
Epoch 8/100
7/7 [==

In [150]:
se_test = model2.evaluate(x_test, y_test)

2/2 [==============================] - 0s 14ms/step - loss: 0.2045 - accuracy: 0.7885


In [151]:
from sklearn import preprocessing
yhat_probs = model2.predict(x_test, verbose=0)
yhat_classes = np.argmax(yhat_probs, axis=1)
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:]
#print(yhat_classes)

yhat_classes = yhat_classes.reshape(-1, 1)
#print(yhat_classes.shape)

ohcc = preprocessing.OneHotEncoder()
outt = ohcc.fit_transform(yhat_classes)

In [152]:
accuracy = accuracy_score(y_test, outt)
print('Accuracy: %f' % accuracy)

Accuracy: 0.788462


In [153]:
print("Classification Report is:\n",classification_report(y_test,outt))

Classification Report is:
               precision    recall  f1-score   support

           0       0.79      0.81      0.80        27
           1       0.79      0.76      0.78        25

   micro avg       0.79      0.79      0.79        52
   macro avg       0.79      0.79      0.79        52
weighted avg       0.79      0.79      0.79        52
 samples avg       0.79      0.79      0.79        52



In [133]:
sc= MinMaxScaler()
p_data=pd.DataFrame(sc.fit_transform(x),columns=x.columns,index=x.index)
x_ne = pdata.iloc[:, :]

In [134]:
x_train, x_test, y_train, y_test = train_test_split(x_ne , y_sex, test_size=0.2 , random_state=1)

In [135]:
y_r = y_sex.to_numpy()
y_r.shape

y_r = y_r.reshape(-1,1)
print(y_r.shape)

encod = OneHotEncoder()
aa =encod.fit(y_r)

y_encod = aa.transform(y_r).toarray()


(239, 1)


In [136]:
from random import randint
x_train, x_test, y_train, y_test = train_test_split(x_ne , y_encod, test_size=0.2 , random_state=randint(100, 1000))
print(x_train.shape ,x_test.shape ,y_train.shape ,y_test.shape)

(191, 12) (48, 12) (191, 2) (48, 2)


In [137]:
model2 = keras.Sequential()

model2.add(Dense(1024, kernel_initializer ="uniform",activation = "relu", input_dim=12))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(512, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(256, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(128, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(64, kernel_initializer = "uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(32, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(16, kernel_initializer ="uniform",activation = "relu"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(8, kernel_initializer ="uniform",activation = "softmax"))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

model2.add(Dense(2))
model2.add(BatchNormalization())
model2.add(Dropout(0.02))

In [138]:
model2.compile(optimizer= "SGD", loss = "MSE", metrics = ["accuracy"])

In [139]:
print("Fit model on training data")
history = model2.fit(x_train, y_train, batch_size=70, epochs=100)

Fit model on training data
Epoch 1/100
3/3 [==============================] - 10s 40ms/step - loss: 1.2537 - accuracy: 0.5969
Epoch 2/100
3/3 [==============================] - 0s 41ms/step - loss: 0.8743 - accuracy: 0.8010
Epoch 3/100
3/3 [==============================] - 0s 62ms/step - loss: 0.7562 - accuracy: 0.8272
Epoch 4/100
3/3 [==============================] - 0s 69ms/step - loss: 0.6239 - accuracy: 0.9058
Epoch 5/100
3/3 [==============================] - 0s 63ms/step - loss: 0.5794 - accuracy: 0.9162
Epoch 6/100
3/3 [==============================] - 0s 40ms/step - loss: 0.4828 - accuracy: 0.9372
Epoch 7/100
3/3 [==============================] - 0s 43ms/step - loss: 0.4930 - accuracy: 0.9110
Epoch 8/100
3/3 [==============================] - 0s 40ms/step - loss: 0.4300 - accuracy: 0.9529
Epoch 9/100
3/3 [==============================] - 0s 41ms/step - loss: 0.3921 - accuracy: 0.9634
Epoch 10/100
3/3 [==============================] - 0s 66ms/step - loss: 0.3567 - accuracy

In [140]:
history = model2.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
6/6 [==============================] - 2s 384ms/step - loss: 0.0436 - accuracy: 0.9686 - val_loss: 0.2478 - val_accuracy: 0.5625
Epoch 2/100
6/6 [==============================] - 0s 50ms/step - loss: 0.0495 - accuracy: 0.9529 - val_loss: 0.2485 - val_accuracy: 0.5625
Epoch 3/100
6/6 [==============================] - 0s 52ms/step - loss: 0.0521 - accuracy: 0.9424 - val_loss: 0.2471 - val_accuracy: 0.5625
Epoch 4/100
6/6 [==============================] - 0s 56ms/step - loss: 0.0406 - accuracy: 0.9791 - val_loss: 0.2470 - val_accuracy: 0.5625
Epoch 5/100
6/6 [==============================] - 0s 48ms/step - loss: 0.0371 - accuracy: 0.9634 - val_loss: 0.2483 - val_accuracy: 0.6250
Epoch 6/100
6/6 [==============================] - 0s 52ms/step - loss: 0.0370 - accuracy: 0.9686 - val_loss: 0.2487 - val_accuracy: 0.6458
Epoch 7/100
6/6 [==============================] - 0s 50ms/step - loss: 0.0402 - accuracy: 0.9791 - val_loss: 0.2483 - val_accuracy: 0.6042
Epoch 8/100
6/6 [==

In [141]:
se_test = model2.evaluate(x_test, y_test)

2/2 [==============================] - 0s 13ms/step - loss: 0.2677 - accuracy: 0.6875


In [142]:
from sklearn import preprocessing
yhat_probs = model2.predict(x_test, verbose=0)
yhat_classes = np.argmax(yhat_probs, axis=1)
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:]
#print(yhat_classes)

yhat_classes = yhat_classes.reshape(-1, 1)
#print(yhat_classes.shape)

ohcc = preprocessing.OneHotEncoder()
outt = ohcc.fit_transform(yhat_classes)

In [143]:
accuracy = accuracy_score(y_test, outt)
print('Accuracy: %f' % accuracy)

Accuracy: 0.687500


In [144]:
print("Classification Report is:\n",classification_report(y_test,outt))

Classification Report is:
               precision    recall  f1-score   support

           0       0.62      0.71      0.67        21
           1       0.75      0.67      0.71        27

   micro avg       0.69      0.69      0.69        48
   macro avg       0.69      0.69      0.69        48
weighted avg       0.70      0.69      0.69        48
 samples avg       0.69      0.69      0.69        48

